In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By

# Khởi tạo EdgeDriver
edge_options = webdriver.EdgeOptions()
edge_options.add_argument("--headless")  # Chạy trình duyệt ẩn danh
edge_options.add_argument("--no-sandbox")
edge_options.add_argument("--disable-dev-shm-usage")
edge_options.add_argument("--disable-gpu")  # Tắt hỗ trợ GPU

# Thêm User-Agent
edge_options.add_argument("--user-agent=Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36")

# Khởi tạo EdgeDriver với các tùy chọn
driver = webdriver.Edge(options=edge_options)

# Thêm User-Agent bằng JavaScript
driver.execute_cdp_cmd("Network.setUserAgentOverride", {"userAgent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36"})

# Chạy trình duyệt ẩn danh
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
print("Đã chạy trình duyệt ẩn thành công!")

Đã chạy trình duyệt ẩn thành công!


In [5]:
import os
import time
from concurrent.futures import ThreadPoolExecutor
from selenium import webdriver
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.edge.options import Options
from bs4 import BeautifulSoup

# Đường dẫn đến thư mục lưu trữ
output_dir = "url_crawled"

# Tạo thư mục nếu chưa tồn tại
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Khởi tạo EdgeDriver
def get_driver():
    edge_options = Options()
    edge_options.use_chromium = True
    edge_options.add_argument("--headless") # Chạy trình duyệt ẩn danh
    edge_options.add_argument("--no-sandbox")
    edge_options.add_argument("--disable-dev-shm-usage")
    edge_options.add_argument("--disable-gpu") # Tắt hỗ trợ GPU
    edge_options.add_argument("--user-agent=Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36")
    
    # Đường dẫn đến edgedriver trên máy của bạn
    edge_driver_path = "C:/EdgeDriver/msedgedriver.exe"
    service = EdgeService(executable_path=edge_driver_path)
    
    return webdriver.Edge(service=service, options=edge_options)

# Hàm để thu thập dữ liệu từ một trang
def extract_and_save_links(page_number):
    try:
        driver = get_driver()
        print(f"Đang lấy dữ liệu từ trang {page_number}...")
        
        driver.get(f'https://batdongsan.com.vn/nha-dat-ban/p{page_number}')
        html_content = driver.page_source
        soup = BeautifulSoup(html_content, 'html.parser')
        product_links = soup.find_all(class_="js__product-link-for-product-id")
        
        # Tạo tệp để lưu liên kết
        file_path = os.path.join(output_dir, f"{page_number}.txt")
        
        output_list = []
        for link in product_links:
            href = link['href']
            output_list.append(href)
            time.sleep(1) # Delay for 1 second
        
        # Ghi các liên kết vào tệp
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write("\n".join(output_list))
        
        print(f"Đã lưu các liên kết từ trang {page_number} vào tệp {file_path}")
    except Exception as e:
        print(f"Có lỗi xảy ra khi xử lý trang {page_number}: {str(e)}")
    finally:
        driver.quit()
        driver = None

total_pages = 10

with ThreadPoolExecutor(max_workers=8) as executor:
    futures = [executor.submit(extract_and_save_links, page_number) for page_number in range(1, total_pages + 1)]

print("Quá trình thu thập dữ liệu đã hoàn tất.")

Đang lấy dữ liệu từ trang 2...
Đang lấy dữ liệu từ trang 8...
Đang lấy dữ liệu từ trang 4...
Đang lấy dữ liệu từ trang 1...
Đang lấy dữ liệu từ trang 6...
Đang lấy dữ liệu từ trang 7...
Đang lấy dữ liệu từ trang 3...
Đang lấy dữ liệu từ trang 5...
Đã lưu các liên kết từ trang 1 vào tệp url_crawled\1.txt
Đang lấy dữ liệu từ trang 9...
Đã lưu các liên kết từ trang 8 vào tệp url_crawled\8.txt
Đã lưu các liên kết từ trang 5 vào tệp url_crawled\5.txt
Đang lấy dữ liệu từ trang 10...
Đã lưu các liên kết từ trang 9 vào tệp url_crawled\9.txt
Đã lưu các liên kết từ trang 2 vào tệp url_crawled\2.txt
Đã lưu các liên kết từ trang 10 vào tệp url_crawled\10.txt
Đã lưu các liên kết từ trang 10 vào tệp url_crawled\10.txt
Đã lưu các liên kết từ trang 7 vào tệp url_crawled\7.txt
Đã lưu các liên kết từ trang 4 vào tệp url_crawled\4.txt
Đã lưu các liên kết từ trang 3 vào tệp url_crawled\3.txt
Có lỗi xảy ra khi xử lý trang 6: Message: timeout: Timed out receiving message from renderer: 297.394
  (Session in

In [24]:
path = "url_crawled"

def read_file_to_list(file_path):
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            lines = file.readlines()
        # Remove newline characters and extra whitespace
        cleaned_lines = [f"https://batdongsan.com.vn/{line.strip()}" for line in lines if line.strip()]
        return cleaned_lines
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")
        return []
        

In [ ]:
import os
import time
from concurrent.futures import ThreadPoolExecutor
from selenium import webdriver
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.edge.options import Options
from bs4 import BeautifulSoup

# Path to the directory where the URLs are stored
path = "url_crawled"

# Path to the directory where the crawled data will be stored
output_dir = "data_crawled"

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Function to initialize the EdgeDriver
def get_driver():
    edge_options = Options()
    edge_options.use_chromium = True
    edge_options.add_argument("--headless")  # Run the browser in headless mode
    edge_options.add_argument("--no-sandbox")
    edge_options.add_argument("--disable-dev-shm-usage")
    edge_options.add_argument("--disable-gpu")  # Disable GPU acceleration
    edge_options.add_argument(
        "user-agent=Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36"
    )

    # Path to the EdgeDriver on your machine
    edge_driver_path = "C:/EdgeDriver/msedgedriver.exe"
    service = EdgeService(executable_path=edge_driver_path)

    return webdriver.Edge(service=service, options=edge_options)

# Function to read a file and return a list of cleaned URLs
def read_file_to_list(file_path):
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            lines = file.readlines()
        cleaned_lines = [f"https://batdongsan.com.vn/{line.strip()}" for line in lines if line.strip()]
        return cleaned_lines
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")
        return []

# Function to process a URL and save the HTML content
def process_url(url, i, k):
    try:
        driver = get_driver()
        print(f"Retrieving data from URL {url}...")

        driver.get(url)
        html_content = driver.page_source

        # Create the file name and save the HTML content to the local machine
        sub_dir = os.path.join(output_dir, str(i))
        if not os.path.exists(sub_dir):
            os.makedirs(sub_dir)
        file_name = os.path.join(sub_dir, f"{k}.html")

        with open(file_name, "w", encoding="utf-8") as file:
            file.write(html_content)
45
4


        driver.quit()
        print(f"Data from URL {url} saved to file {file_name}")
        return True
    except Exception as e:
        print("Error:", e)
        return False

# Loop to read through each .txt file in the directory
for i in range(1, 11):
    urls = read_file_to_list(os.path.join(path, f"{i}.txt"))

    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = []
        for url in urls:
            future = executor.submit(process_url, url, i, len(futures) + 1)
            futures.append(future)

print("Processing completed.")

Retrieving data from URL https://batdongsan.com.vn//ban-shophouse-nha-pho-thuong-mai-duong-nghiem-xuan-yem-phuong-dai-kim-prj-the-manor-central-park/ban-moi-nhat-dt-nguyen-xien-mat-lo-hoang-lh-pr39928165...
Retrieving data from URL https://batdongsan.com.vn//ban-can-ho-chung-cu-phuong-tay-mo-prj-imperia-sola-park/-booking-dot-1-ck-16-1pn-1-gia-2-09ty-2pn-gia-2-55ty-3pn-gia-3-7ty-htls-30-thang-pr39915391...
Retrieving data from URL https://batdongsan.com.vn//ban-can-ho-chung-cu-phuong-long-thanh-my-prj-vinhomes-grand-park/hot-gh-ch-beverly-solari-gia-re-chi-1-499-ty-lh-0938703286-pr33552131...
Retrieving data from URL https://batdongsan.com.vn//ban-dat-xa-minh-tri/ban-tho-cu-co-so-do-dt-tu-100-5000m-gan-khu-du-lich-ho-dong-do-soc-son-hn-pr39441666...
Retrieving data from URL https://batdongsan.com.vn//ban-can-ho-chung-cu-phuong-tay-mo-prj-vinhomes-smart-city/ban-3pn-bc-dong-nam-view-be-boi-5-ty-truc-tiep-cdt-ck-8-vay-0-den-t8-2026-pr39904136...
Data from URL https://batdongsan.com.vn//b